In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 한글폰트 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumGothic')
sns.set(font="NanumGothic",#"NanumGothicCoding",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

# 경고 표시 무시
import warnings
warnings.filterwarnings('ignore')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 3s (3,689 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120874 files and dire

In [2]:
# 출력 옵션 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# 구글 코랩 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# train 데이터 불러오기
train_data = pd.read_csv('/content/drive/MyDrive/미니프로젝트5차part2_Kaggle/train.csv')

In [5]:
train_data.shape

(100000, 9)

In [6]:
train_data['label'].value_counts()

2     12170
8     11497
1     11465
7     10594
10     9579
9      9282
11     9112
3      8305
4      7208
5      5411
6      5377
Name: label, dtype: int64

* 클래스 불균형이 존재한다고 판단할 수 있다. 따라서 우리는 클래스 불균형을 방지하고 클래스 균형을 이끌도록 해야 한다.

* 다음은 클래스 불균형 처리를 해결하기 위한 방안이다.


<img src = 'https://drive.google.com/uc?id=1PxMuiKu4Z8qcdDW27JUb_chNfTm3dq1d' height = 300 width = 600>

* 클래스 불균형을 처리하기 위한 일반적인 방법은 리샘플링(Resampling) 방법이지만<br>
여기서는 가중치 조정을 이용하는 방법을 이용하고자 한다.

* 다음은 머신러닝, 딥러닝에서 가중치 조정을 이용하는 방법을 알려준다.


<img src = 'https://drive.google.com/uc?id=1wPXWIUj4I1CN7SBNyPZnF3RzSldgn56Q' height = 300 width = 600>

<img src = 'https://drive.google.com/uc?id=1GqoyQBTM5UgZOzZkNlr1G3jm-uOHZXxg' height = 300 width = 600>



* 머신러닝, 딥러닝 사용할 떄 class_weights 정보를 부여하면 된다.

* 하지만 지금 클래스 불균형을 처리하기 보다 모델 1, 모델 2-1, 모델 2-2를 만들 떄 그떄 되서 클래스 불균형을 확인하여<br> 만약 존재하면 가중치를 더 부여하는 방법으로 클래스 불균형을 해결하고자 한다.

In [7]:
# train 데이터에 대해 전반적인 탐색을 해야겠다.
train_data.head()

,Unnamed: 0,timestamp,A_x,A_y,A_z,B_x,B_y,B_z,label
0,0,2019-01-12 00:45:54.450,-0.259130,-0.834869,-0.485499,0.196409,NaN,0.384934,8
1,1,2000-01-01 01:37:06.440,0.370490,0.175042,0.122625,-0.338242,0.358245,0.126491,2
2,2,2019-01-12 00:45:33.900,-0.257837,-0.881947,-0.391895,0.196027,0.894537,0.411221,8
3,3,2000-01-01 00:46:22.680,-0.937753,-0.055961,0.362041,-0.929881,0.087673,0.134609,11
4,4,2000-01-01 00:49:56.620,-0.988320,-0.190390,0.157909,-0.954669,-0.024810,-0.388420,6


In [8]:
train_data.tail()

,Unnamed: 0,timestamp,A_x,A_y,A_z,B_x,B_y,B_z,label
99995,99995,2000-01-01 00:25:51.300,-0.499562,0.012127,0.365746,NaN,-0.040284,-0.180426,10
99996,99996,2000-01-01 01:37:53.120,-0.929146,NaN,-0.362481,-0.951160,-0.047168,-0.344213,3
99997,99997,2000-01-01 00:01:02.020,-0.940124,-0.301950,NaN,-0.983245,-0.011691,-0.178657,6
99998,99998,2000-01-01 00:31:54.960,-1.152895,-0.149863,-0.746005,-0.945021,-0.686593,0.317497,5
99999,99999,2000-01-01 00:44:12.440,-0.851409,-0.123314,0.409099,-0.862848,-0.135681,-0.013041,11


In [9]:
# train 데이터에 'Unamed : 0' 삭제한다.
train_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [10]:
train_data['timestamp'] = pd.to_datetime(train_data['timestamp']) # 'timestamp' 변수의 데이터타입을 datetime으로 변경

* train_data는 시계열 데이터인데, 시간순으로 정렬되어 있지 않아서 오름차순으로 정렬해야 한다.

In [11]:
train_data = train_data.sort_values(by='timestamp', ) # 'timestamp' 변수를 오름차순 정렬한다. 진정한 시계열 데이터에 맞게

In [12]:
# index 번호를 다시 0부터 부여한다. (왜냐하면 'timestamp' 변수를 오름차순 정렬했기 떄문에 index 번호가 불규칙적으로 배치될 것이기 떄문이다.)
train_data = train_data.reset_index(drop=True)

In [13]:
train_data.head()

,timestamp,A_x,A_y,A_z,B_x,B_y,B_z,label
0,2000-01-01 00:00:00.000,-0.988758,-0.144844,0.183325,-0.991851,-0.022659,-0.306466,6
1,2000-01-01 00:00:00.020,-1.008465,NaN,0.156743,-0.976224,-0.040058,-0.277258,6
2,2000-01-01 00:00:00.040,-0.991936,-0.143049,0.156618,-0.949758,-0.035796,-0.299548,6
3,2000-01-01 00:00:00.060,-1.007975,-0.140831,0.155302,-1.006806,-0.013057,-0.289020,6
4,2000-01-01 00:00:00.080,-0.991991,-0.159979,0.166965,NaN,0.015054,-0.293883,6


* 시계열 데이터는 0.2, 0.4초 정도 각각 차이 나는 것 같다.
  * 나는 그점이 우려스러웠다. 뭐냐면 갑자기 시간 차이가 많이 날떄(ex.갑자기 2분, 3분, 30분 ...) 가 존재할 수가 있으면<br> 어떻게 결측치를 처리할까 그런 고민을 한 적이 있었다.

* 시계열 데이터에 대한 결측치(NaN)을 어떻게 처리하느냐가 관건이다.
  * 'A_x' ~ 'B_z' 까지 결측치가 10000개 있어서 이를 어떻게 처리하느냐가 관건이다.

* 전략) 저는 결측치 기준으로 이전 값과 이후 값을 파악해서 평균 구한 값으로 대체하려고 합니다.

* 하지만 그전에 해야할일이 있다. 바로 시간대 정보를 결합해서 새로운 변수 'Combined'를 만드는 것이다.

In [13]:
train_data.info() # 변수에 대한 결측치를 확인한다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  100000 non-null  datetime64[ns]
 1   A_x        90000 non-null   float64       
 2   A_y        90000 non-null   float64       
 3   A_z        90000 non-null   float64       
 4   B_x        90000 non-null   float64       
 5   B_y        90000 non-null   float64       
 6   B_z        90000 non-null   float64       
 7   label      100000 non-null  int64         
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 6.1 MB


* 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second', 'MicroSecond', 'Combined' 변수를 추가한다.

In [14]:
# Timestamp 열의 각 구성 요소를 추출
train_data['Year'] = train_data['timestamp'].dt.year.astype(str)
train_data['Month'] = train_data['timestamp'].dt.month.astype(str).str.zfill(2)
train_data['Day'] = train_data['timestamp'].dt.day.astype(str).str.zfill(2)
train_data['Hour'] = train_data['timestamp'].dt.hour.astype(str).str.zfill(2)
train_data['Minute'] = train_data['timestamp'].dt.minute.astype(str).str.zfill(2)
train_data['Second'] = train_data['timestamp'].dt.second.astype(str).str.zfill(2)
train_data['MicroSecond'] = (train_data['timestamp'].dt.microsecond // 1000).astype(str).str.zfill(3)

# 각 구성 요소를 하나의 정수로 결합
train_data['Combined'] = (train_data['Year'] + train_data['Month'] + train_data['Day'] + train_data['Hour'] + train_data['Minute'] + train_data['Second'] + train_data['MicroSecond']).astype(int)
# train_data['Combined'] = train_data['Year'] * 10000000000 + train_data['Month'] * 100000000 + train_data['Day'] * 1000000 + train_data['Hour'] * 10000 + train_data['Minute'] * 100 + train_data['Second']

In [15]:
# 'Year'부터 'MicroSecond' 변수 삭제
# train_data.drop(['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second', 'MicroSecond'], axis=1, inplace=True)

In [16]:
# 'Year', 'Month' ~ 'MicroSecond' 변수의 데이터타입을 int로 변환
train_data['Year'] = train_data['Year'].astype(int)
train_data['Month'] = train_data['Month'].astype(int)
train_data['Day'] = train_data['Day'].astype(int)
train_data['Hour'] = train_data['Hour'].astype(int)
train_data['Minute'] = train_data['Minute'].astype(int)
train_data['Second'] = train_data['Second'].astype(int)
train_data['MicroSecond'] = train_data['MicroSecond'].astype(int)

In [17]:
train_data.head()

,timestamp,A_x,A_y,A_z,B_x,B_y,B_z,label,Year,Month,Day,Hour,Minute,Second,MicroSecond,Combined
0,2000-01-01 00:00:00.000,-0.988758,-0.144844,0.183325,-0.991851,-0.022659,-0.306466,6,2000,1,1,0,0,0,0,20000101000000000
1,2000-01-01 00:00:00.020,-1.008465,NaN,0.156743,-0.976224,-0.040058,-0.277258,6,2000,1,1,0,0,0,20,20000101000000020
2,2000-01-01 00:00:00.040,-0.991936,-0.143049,0.156618,-0.949758,-0.035796,-0.299548,6,2000,1,1,0,0,0,40,20000101000000040
3,2000-01-01 00:00:00.060,-1.007975,-0.140831,0.155302,-1.006806,-0.013057,-0.289020,6,2000,1,1,0,0,0,60,20000101000000060
4,2000-01-01 00:00:00.080,-0.991991,-0.159979,0.166965,NaN,0.015054,-0.293883,6,2000,1,1,0,0,0,80,20000101000000080


In [18]:
train_data.set_index('timestamp', inplace=True) # 'timestamp' 열을 인덱스로 설정

In [19]:
train_data.head(10) # 'timestamp'열이 인덱스로 설정됐는지 재확인한다.

,A_x,A_y,A_z,B_x,B_y,B_z,label,Year,Month,Day,Hour,Minute,Second,MicroSecond,Combined
timestamp,,,,,,,,,,,,,,,
2000-01-01 00:00:00.000,-0.988758,-0.144844,0.183325,-0.991851,-0.022659,-0.306466,6,2000,1,1,0,0,0,0,20000101000000000
2000-01-01 00:00:00.020,-1.008465,NaN,0.156743,-0.976224,-0.040058,-0.277258,6,2000,1,1,0,0,0,20,20000101000000020
2000-01-01 00:00:00.040,-0.991936,-0.143049,0.156618,-0.949758,-0.035796,-0.299548,6,2000,1,1,0,0,0,40,20000101000000040
2000-01-01 00:00:00.060,-1.007975,-0.140831,0.155302,-1.006806,-0.013057,-0.289020,6,2000,1,1,0,0,0,60,20000101000000060
2000-01-01 00:00:00.080,-0.991991,-0.159979,0.166965,NaN,0.015054,-0.293883,6,2000,1,1,0,0,0,80,20000101000000080
2000-01-01 00:00:00.100,-1.008148,-0.152922,0.171494,-0.967190,-0.041271,-0.289058,6,2000,1,1,0,0,0,100,20000101000000100
2000-01-01 00:00:00.120,-0.991536,-0.159341,0.160002,-0.936596,-0.070587,-0.294263,6,2000,1,1,0,0,0,120,20000101000000120
2000-01-01 00:00:00.180,-0.990727,-0.146829,NaN,-0.967847,-0.017197,-0.275129,6,2000,1,1,0,0,0,180,20000101000000180
2000-01-01 00:00:00.200,-0.976904,-0.131678,0.182595,-0.984224,-0.017629,-0.257675,6,2000,1,1,0,0,0,200,20000101000000200


In [20]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100000 entries, 2000-01-01 00:00:00 to 2019-01-12 02:02:02.100000
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   A_x          90000 non-null   float64
 1   A_y          90000 non-null   float64
 2   A_z          90000 non-null   float64
 3   B_x          90000 non-null   float64
 4   B_y          90000 non-null   float64
 5   B_z          90000 non-null   float64
 6   label        100000 non-null  int64  
 7   Year         100000 non-null  int64  
 8   Month        100000 non-null  int64  
 9   Day          100000 non-null  int64  
 10  Hour         100000 non-null  int64  
 11  Minute       100000 non-null  int64  
 12  Second       100000 non-null  int64  
 13  MicroSecond  100000 non-null  int64  
 14  Combined     100000 non-null  int64  
dtypes: float64(6), int64(9)
memory usage: 12.2 MB


In [21]:
# 결측치를 전 값과 후 값의 평균으로 대체
train_data[['A_x', 'A_y','A_z', 'B_x', 'B_y','B_z']] = (train_data[['A_x', 'A_y',	'A_z', 'B_x', 'B_y','B_z']].fillna(method='ffill') + train_data[['A_x', 'A_y',	'A_z', 'B_x', 'B_y','B_z']].fillna(method='bfill')) / 2

In [22]:
# train_data.interpolate(linear='time', inplace=True) # 결측치를 (이전 값 +  이후 값) / 2로 반영

In [23]:
train_data.head() # 결측치가 잘 대체됐는지 확인한다.

,A_x,A_y,A_z,B_x,B_y,B_z,label,Year,Month,Day,Hour,Minute,Second,MicroSecond,Combined
timestamp,,,,,,,,,,,,,,,
2000-01-01 00:00:00.000,-0.988758,-0.144844,0.183325,-0.991851,-0.022659,-0.306466,6,2000,1,1,0,0,0,0,20000101000000000
2000-01-01 00:00:00.020,-1.008465,-0.143947,0.156743,-0.976224,-0.040058,-0.277258,6,2000,1,1,0,0,0,20,20000101000000020
2000-01-01 00:00:00.040,-0.991936,-0.143049,0.156618,-0.949758,-0.035796,-0.299548,6,2000,1,1,0,0,0,40,20000101000000040
2000-01-01 00:00:00.060,-1.007975,-0.140831,0.155302,-1.006806,-0.013057,-0.289020,6,2000,1,1,0,0,0,60,20000101000000060
2000-01-01 00:00:00.080,-0.991991,-0.159979,0.166965,-0.986998,0.015054,-0.293883,6,2000,1,1,0,0,0,80,20000101000000080


In [24]:
# 다시 결측치를 확인해본다. 결측치가 대체되었는지 재확인한다.
train_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100000 entries, 2000-01-01 00:00:00 to 2019-01-12 02:02:02.100000
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   A_x          100000 non-null  float64
 1   A_y          100000 non-null  float64
 2   A_z          100000 non-null  float64
 3   B_x          100000 non-null  float64
 4   B_y          100000 non-null  float64
 5   B_z          100000 non-null  float64
 6   label        100000 non-null  int64  
 7   Year         100000 non-null  int64  
 8   Month        100000 non-null  int64  
 9   Day          100000 non-null  int64  
 10  Hour         100000 non-null  int64  
 11  Minute       100000 non-null  int64  
 12  Second       100000 non-null  int64  
 13  MicroSecond  100000 non-null  int64  
 14  Combined     100000 non-null  int64  
dtypes: float64(6), int64(9)
memory usage: 12.2 MB


* 시계열 데이터를 반영해 결측치를 제거했으니 이제 'timestamp' 변수는 삭제한다.

In [25]:
# 인덱스 재설정
train_data.reset_index(drop=True, inplace=True)

In [26]:
train_data.head()

,A_x,A_y,A_z,B_x,B_y,B_z,label,Year,Month,Day,Hour,Minute,Second,MicroSecond,Combined
0,-0.988758,-0.144844,0.183325,-0.991851,-0.022659,-0.306466,6,2000,1,1,0,0,0,0,20000101000000000
1,-1.008465,-0.143947,0.156743,-0.976224,-0.040058,-0.277258,6,2000,1,1,0,0,0,20,20000101000000020
2,-0.991936,-0.143049,0.156618,-0.949758,-0.035796,-0.299548,6,2000,1,1,0,0,0,40,20000101000000040
3,-1.007975,-0.140831,0.155302,-1.006806,-0.013057,-0.289020,6,2000,1,1,0,0,0,60,20000101000000060
4,-0.991991,-0.159979,0.166965,-0.986998,0.015054,-0.293883,6,2000,1,1,0,0,0,80,20000101000000080


In [27]:
train_data.info() # 데이터타입이 잘 변경됐는지 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   A_x          100000 non-null  float64
 1   A_y          100000 non-null  float64
 2   A_z          100000 non-null  float64
 3   B_x          100000 non-null  float64
 4   B_y          100000 non-null  float64
 5   B_z          100000 non-null  float64
 6   label        100000 non-null  int64  
 7   Year         100000 non-null  int64  
 8   Month        100000 non-null  int64  
 9   Day          100000 non-null  int64  
 10  Hour         100000 non-null  int64  
 11  Minute       100000 non-null  int64  
 12  Second       100000 non-null  int64  
 13  MicroSecond  100000 non-null  int64  
 14  Combined     100000 non-null  int64  
dtypes: float64(6), int64(9)
memory usage: 11.4 MB


<h1> 본격적인 모델링을 시작합시다. </h1>

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

* 우리 이런 흐름으로 갈 것입니다. 물론 label이 11개여서 아주 세세하게 따지면 다른 부분이 존재하겠죠?

* 하지만 우리는 이런 흐름으로 갈 거예요

* 정적행동(0), 동적행동(1)으로 나눈다.

* label을 바탕으로 저는 이렇게 동적 행동(1)과 정적 행동(1)으로 구분하였습니다.
  * 1: 걷기 -> 동적 행동(1)로 정의
  * 2. 뛰기 -> 동적 행동(1)로 정의
  * 3. 천천히 걷기 -> 동적 행동(1)로 정의
  * 4. 계단 오르기 -> 동적 행동(1)로 정의
  * 5. 계단 내려가기 -> 동적 행동(1)로 정의
  * 6. 서있기 -> 정적 행동(0)으로 정의
  * 7. 앉아있기 -> 정적 행동(0)으로 정의
  * 8. 누워있기 -> 정적 행동(0)으로 정의
  * 9. 자전거 타기 -> 동적 행동(1)로 정의
  * 10. 서서 자전거 타기 -> 동적 행동(1)로 정의
  * 11. 자전거에 앉아 있기 -> 정적 행동(0)으로 정의

In [28]:
# is_dynamic 변수 추가
train_data['is_dynamic'] = train_data['label'].apply(lambda label: 0 if label in [6, 7, 8, 11] else 1)

* 우리 train_data에서 X, y1(label), y2(is_dynamic)를 구분해요

In [29]:
train_X = train_data[['A_x', 'A_y',	'A_z', 'B_x', 'B_y', 'B_z', 'Year',	'Month', 'Day',	'Hour',	'Minute','Second',	'MicroSecond', 'Combined']]
train_y1 = train_data['label']
train_y2 = train_data['is_dynamic']

* 표준화를 진행한다.
  * MinMax말고 StandardScaler를 사용한다.
  * 일단 학습 데이터만 표준화를 진행하는 것처럼 보이겠지만
  * 나중에 추가로 테스트 데이터도 표준화할 예정입니다.

In [30]:
from sklearn.preprocessing import StandardScaler

In [31]:
columns = train_X.columns

In [32]:
sds = StandardScaler()
train_X= pd.DataFrame(sds.fit_transform(train_X), columns = columns) # 저는 2차원 넘파이 배열로 만들기 싫어서 데이터프레임으로 만들겠습니다.

In [33]:
train_X.head()

,A_x,A_y,A_z,B_x,B_y,B_z,Year,Month,Day,Hour,Minute,Second,MicroSecond,Combined
0,-0.243301,-0.099529,0.545522,-0.263151,-0.238577,-0.613908,-0.67613,0.0,-0.67613,-0.681911,-1.573525,-1.664163,-1.698947,-0.676131
1,-0.286139,-0.096934,0.471650,-0.243766,-0.268530,-0.576119,-0.67613,0.0,-0.67613,-0.681911,-1.573525,-1.664163,-1.629705,-0.676131
2,-0.250208,-0.094339,0.471302,-0.210936,-0.261193,-0.604958,-0.67613,0.0,-0.67613,-0.681911,-1.573525,-1.664163,-1.560464,-0.676131
3,-0.285073,-0.087925,0.467644,-0.281701,-0.222047,-0.591336,-0.67613,0.0,-0.67613,-0.681911,-1.573525,-1.664163,-1.491223,-0.676131
4,-0.250328,-0.143283,0.500057,-0.257131,-0.173656,-0.597628,-0.67613,0.0,-0.67613,-0.681911,-1.573525,-1.664163,-1.421982,-0.676131


<h2> 정적 행동(0)과 동적 행동(1)를 구분하는 모델1를 만들어요! </h2>

* 0인지 1인지를 구분하는 곳에서 클래스 균형인지 클래스 불균형인지 확인한다.

In [34]:
train_y2.value_counts()

1    63420
0    36580
Name: is_dynamic, dtype: int64

* 클래스 불균형인 것 같다. 따라서 0인 부분에 대해서 가중치를 더 부여해야 할 것 같다.

In [35]:
from sklearn.utils.class_weight import compute_class_weight

In [36]:
# 클래스 레이블과 해당 레이블의 빈도수
labels = np.unique(train_y2)
class_isdynamic_weights = compute_class_weight(class_weight='balanced', classes=labels, y=train_y2)

# 클래스 가중치 딕셔너리 생성
class_isdynamic_weight_dict = {label: weight for label, weight in zip(labels, class_isdynamic_weights)}

In [37]:
class_isdynamic_weight_dict # 0를 가중치 더 부여하는 것을 알 수 있다.

{0: 1.366867140513942, 1: 0.7883948281299274}

* 0인 것을 가중치 더 부여해서 모델1를 훈련한다.

In [38]:
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgbm

In [39]:
# 다양한 분류기 생성
lr1 = LogisticRegression(class_weight = class_isdynamic_weight_dict, random_state=42)
rf1= RandomForestClassifier(class_weight = class_isdynamic_weight_dict, random_state=42)
dt1 = DecisionTreeClassifier(class_weight = class_isdynamic_weight_dict, random_state=42)
svc1 = SVC(probability=True, class_weight = class_isdynamic_weight_dict, random_state=42)
etc1 = ExtraTreesClassifier(n_jobs=-1, class_weight = class_isdynamic_weight_dict, random_state=42)
hgbc1 = HistGradientBoostingClassifier(class_weight=class_isdynamic_weight_dict, random_state=42)
lgbm1 = lgbm.LGBMClassifier(class_weight=class_isdynamic_weight_dict, random_state=42)

# VotingClassifier 생성
model1 = VotingClassifier(estimators=[('lr1', lr1), ('rf1', rf1), ('dt1', dt1), ('svc1', svc1), ('etc1', etc1), ('hgbc1', hgbc1), ('lgbm1', lgbm1)], voting='hard', n_jobs=-1,)
model1.fit(train_X, train_y2)

VotingClassifier(estimators=[('lr1',
                              LogisticRegression(class_weight={0: 1.366867140513942,
                                                               1: 0.7883948281299274},
                                                 random_state=42)),
                             ('rf1',
                              RandomForestClassifier(class_weight={0: 1.366867140513942,
                                                                   1: 0.7883948281299274},
                                                     random_state=42)),
                             ('dt1',
                              DecisionTreeClassifier(class_weight={0: 1.366867140513942,
                                                                   1: 0.7883948281299274},
                                                     random_state=42)),
                             ('svc1',
                              SVC(c...
                                  probability=True, random_state=42)),
                             ('etc1',
                              ExtraTreesClassifier(class_weight={0: 1.366867140513942,
                                                                 1: 0.7883948281299274},
                                                   n_jobs=-1,
                                                   random_state=42)),
                             ('hgbc1',
                              HistGradientBoostingClassifier(class_weight={0: 1.366867140513942,
                                                                           1: 0.7883948281299274},
                                                             random_state=42)),
                             ('lgbm1',
                              LGBMClassifier(class_weight={0: 1.366867140513942,
                                                           1: 0.7883948281299274},
                                             random_state=42))],
                 n_jobs=-1)

<h2> 정적 행동(0)으로 판단했으면 정적 행동 중에서 어떤 정적행동(6, 7, 8, 11) 인지 구분하는 모델 2-1를 만들어요! </h2>

In [40]:
# train_y1에서 정적 행동(6, 7, 8, 11)에 해당하는 행들의 인덱스를 찾기
indices = train_y1[train_y1.isin([6, 7, 8, 11])].index

# train_y1에서 정적 행동(6, 7, 8, 11)에 해당하는 것들만 모으기
static_train_y = train_y1.loc[indices]

# 해당하는 행들만 train_X에서 선택
static_train_X = train_X.loc[indices]

In [41]:
# static_train_X, static_train_y index를 0부터 다시 배치한다. 왜냐하면 index 번호가 불규칙적이게 되어있기 떄문
static_train_X = static_train_X.reset_index(drop=True)
static_train_y = static_train_y.reset_index(drop=True)

* static_train_y에 대해서 클래스 균형인지 불균형인지 확인한다.
  * 만약 불균형이라면 소수 클래스에 대해서 가중치를 더 부여하는 식으로 방향을 잡아야 한다.

In [42]:
static_train_y.value_counts() # 클래스 불균형이다.

8     11497
7     10594
11     9112
6      5377
Name: label, dtype: int64

* 정적 행동(6, 7, 8, 11)를 각각 0, 1, 2, 3으로 변환한다.
  * Scikit-learn의 많은 분류기들은 내부적으로 y 레이블을 0부터 시작하는 연속적인 정수로 자동으로 변환해주는데

    하지만 VotingClassifier는 이러한 변환을 해주지 않기 떄문에 수동적으로 우리가 y 레이블을 0부터 시작하는 연속적인 정수로 만들어줘야 한다.


In [43]:
from sklearn.utils.class_weight import compute_class_weight

In [44]:
# 클래스 레이블과 해당 레이블의 빈도수
labels = np.unique(static_train_y)
class_staticClassification_weights = compute_class_weight(class_weight='balanced', classes=labels, y=static_train_y)

# 클래스 가중치 딕셔너리 생성
class_staticClassification_weight_dict = {label: weight for label, weight in zip(labels, class_staticClassification_weights)}

In [45]:
class_staticClassification_weight_dict # 6를 가중치 더 부여하는 것을 알 수 있다.

{6: 1.7007625069741492,
 7: 0.8632244666792525,
 8: 0.7954248934504653,
 11: 1.0036215978928884}

In [46]:
from sklearn.preprocessing import LabelEncoder

# le.classes_하면 6, 7, 8, 11이 print 된다.
le = LabelEncoder()
static_train_y_encoded = pd.Series(le.fit_transform(static_train_y)) # 1차원 넘파이 배열보다는 Series 형태로 사용하는 것을 선호해서 변환해준다.
class_staticClassification_weight_encoded_dict= {i: class_staticClassification_weight_dict[class_label] for i, class_label in enumerate(le.classes_)}

In [47]:
class_staticClassification_weight_encoded_dict

{0: 1.7007625069741492,
 1: 0.8632244666792525,
 2: 0.7954248934504653,
 3: 1.0036215978928884}

In [48]:
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgbm

In [49]:
# 다양한 분류기 생성
lr2_1 = LogisticRegression(class_weight = class_staticClassification_weight_encoded_dict, random_state=42)
rf2_1= RandomForestClassifier(class_weight = class_staticClassification_weight_encoded_dict, random_state=42)
dt2_1 = DecisionTreeClassifier(class_weight = class_staticClassification_weight_encoded_dict, random_state=42)
svc2_1 = SVC(probability=True, class_weight = class_staticClassification_weight_encoded_dict, random_state=42)
etc2_1 = ExtraTreesClassifier(n_jobs=-1, class_weight = class_staticClassification_weight_encoded_dict, random_state=42)
hgbc2_1 = HistGradientBoostingClassifier(class_weight = class_staticClassification_weight_encoded_dict, random_state=42)
lgbm2_1 = lgbm.LGBMClassifier(class_weight=class_staticClassification_weight_encoded_dict, random_state=42)

# VotingClassifier 생성
model2_1 = VotingClassifier(estimators=[('lr2_1', lr2_1), ('rf2_1', rf2_1), ('dt2_1', dt2_1), ('svc2_1', svc2_1), ('etc2_1', etc2_1), ('hgbc2_1', hgbc2_1), ('lgbm2_1', lgbm2_1)], voting='hard', n_jobs=-1,)
model2_1.fit(static_train_X, static_train_y_encoded)

VotingClassifier(estimators=[('lr2_1',
                              LogisticRegression(class_weight={0: 1.7007625069741492,
                                                               1: 0.8632244666792525,
                                                               2: 0.7954248934504653,
                                                               3: 1.0036215978928884},
                                                 random_state=42)),
                             ('rf2_1',
                              RandomForestClassifier(class_weight={0: 1.7007625069741492,
                                                                   1: 0.8632244666792525,
                                                                   2: 0.7954248934504653,
                                                                   3: 1.0036215978928884},
                                                     random_state=42)),
                             ('dt2_1',
                              DecisionTreeClassi...
                                                                 3: 1.0036215978928884},
                                                   n_jobs=-1,
                                                   random_state=42)),
                             ('hgbc2_1',
                              HistGradientBoostingClassifier(class_weight={0: 1.7007625069741492,
                                                                           1: 0.8632244666792525,
                                                                           2: 0.7954248934504653,
                                                                           3: 1.0036215978928884},
                                                             random_state=42)),
                             ('lgbm2_1',
                              LGBMClassifier(class_weight={0: 1.7007625069741492,
                                                           1: 0.8632244666792525,
                                                           2: 0.7954248934504653,
                                                           3: 1.0036215978928884},
                                             random_state=42))],
                 n_jobs=-1)

<h2> 동적 행동(1)로 판단했으면 동적 행동 중에서 어떤 동적 행동(1, 2, 3, 4, 5, 9, 10)을 구분하는지에 대한 모델 2-2를 만들어요! </h2>

In [50]:
# train_y1에서 동적 행동(1, 2, 3, 4, 5, 9, 10)에 해당하는 행들의 인덱스를 찾기
indices = train_y1[train_y1.isin([1, 2, 3, 4, 5, 9, 10])].index

# train_y1에서 동적 행동(1, 2, 3, 4, 5, 9, 10)에 해당하는 것들만 모으기
dynamic_train_y = train_y1.loc[indices]

# 해당하는 행들만 train_X에서 선택
dynamic_train_X = train_X.loc[indices]

In [51]:
# dynamic_train_X, dynamic_train_y index를 0부터 다시 배치한다. 왜냐하면 index 번호가 불규칙적이게 되어있기 떄문
dynamic_train_X = dynamic_train_X.reset_index(drop=True)
dynamic_train_y = dynamic_train_y.reset_index(drop=True)

* dynamic_train_y에 대해서 클래스 균형인지 불균형인지 확인한다.
  * 만약 불균형이라면 소수 클래스에 대해서 가중치를 더 부여하는 식으로 방향을 잡아야 한다.

In [52]:
dynamic_train_y.value_counts() # 클래스 불균형이다.

2     12170
1     11465
10     9579
9      9282
3      8305
4      7208
5      5411
Name: label, dtype: int64

* 동적 행동(1, 2, 3, 4, 5, 9, 10)를 각각 0, 1, 2, 3, 4, 5, 6으로 변환한다.
  * Scikit-learn의 많은 분류기들은 내부적으로 y 레이블을 0부터 시작하는 연속적인 정수로 자동으로 변환해주는데

    하지만 VotingClassifier는 이러한 변환을 해주지 않기 떄문에 수동적으로 우리가 y 레이블을 0부터 시작하는 연속적인 정수로 만들어줘야 한다.

In [53]:
from sklearn.utils.class_weight import compute_class_weight

In [54]:
# 클래스 레이블과 해당 레이블의 빈도수
labels = np.unique(dynamic_train_y)
class_dynamicClassification_weights = compute_class_weight(class_weight='balanced', classes=labels, y=dynamic_train_y)

# 클래스 가중치 딕셔너리 생성
class_dynamicClassification_weight_dict = {label: weight for label, weight in zip(labels, class_dynamicClassification_weights)}

In [55]:
class_dynamicClassification_weight_dict # 5를 더 가중치 부여하는 것으로 알 수 있다.

{1: 0.7902311382468382,
 2: 0.7444535743631882,
 3: 1.0909090909090908,
 4: 1.2569367369589346,
 5: 1.6743670301238218,
 9: 0.9760827407886231,
 10: 0.9458189790165988}

In [56]:
from sklearn.preprocessing import LabelEncoder

# ld.classes_하면 1, 2, 3, 4, 5, 9, 10이 print 된다.
ld = LabelEncoder()
dynamic_train_y_encoded = pd.Series(ld.fit_transform(dynamic_train_y)) # 1차원 넘파이 배열보다는 Series 형태로 사용하는 것을 선호해서 변환해준다.
class_dynamicClassification_weight_encoded_dict= {i: class_dynamicClassification_weight_dict[class_label] for i, class_label in enumerate(ld.classes_)}

In [57]:
class_dynamicClassification_weight_encoded_dict

{0: 0.7902311382468382,
 1: 0.7444535743631882,
 2: 1.0909090909090908,
 3: 1.2569367369589346,
 4: 1.6743670301238218,
 5: 0.9760827407886231,
 6: 0.9458189790165988}

In [58]:
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgbm

In [59]:
# 다양한 분류기 생성
lr2_2 = LogisticRegression(class_weight = class_dynamicClassification_weight_encoded_dict, random_state=42)
rf2_2= RandomForestClassifier(class_weight = class_dynamicClassification_weight_encoded_dict, random_state=42)
dt2_2 = DecisionTreeClassifier(class_weight = class_dynamicClassification_weight_encoded_dict, random_state=42)
svc2_2 = SVC(probability=True, class_weight = class_dynamicClassification_weight_encoded_dict, random_state=42)
etc2_2 = ExtraTreesClassifier(n_jobs=-1, class_weight = class_dynamicClassification_weight_encoded_dict, random_state=42)
hgbc2_2 = HistGradientBoostingClassifier(class_weight=class_dynamicClassification_weight_encoded_dict, random_state=42)
lgbm2_2 = lgbm.LGBMClassifier(class_weight=class_dynamicClassification_weight_encoded_dict, random_state=42)

# VotingClassifier 생성
model2_2 = VotingClassifier(estimators=[('lr2_2', lr2_2), ('rf2_2', rf2_2), ('dt2_2', dt2_2), ('svc2_2', svc2_2), ('etc2_2', etc2_2), ('hgbc2_2', hgbc2_2), ('lgbm2_2', lgbm2_2)], voting='hard', n_jobs=-1,)
model2_2.fit(dynamic_train_X, dynamic_train_y_encoded)

VotingClassifier(estimators=[('lr2_2',
                              LogisticRegression(class_weight={0: 0.7902311382468382,
                                                               1: 0.7444535743631882,
                                                               2: 1.0909090909090908,
                                                               3: 1.2569367369589346,
                                                               4: 1.6743670301238218,
                                                               5: 0.9760827407886231,
                                                               6: 0.9458189790165988},
                                                 random_state=42)),
                             ('rf2_2',
                              RandomForestClassifier(class_weight={0: 0.7902311382468382,
                                                                   1: 0.7444535743631882,
                                                                   2: 1.0909090909090908,
                                                                   3: 1....
                                                                           1: 0.7444535743631882,
                                                                           2: 1.0909090909090908,
                                                                           3: 1.2569367369589346,
                                                                           4: 1.6743670301238218,
                                                                           5: 0.9760827407886231,
                                                                           6: 0.9458189790165988},
                                                             random_state=42)),
                             ('lgbm2_2',
                              LGBMClassifier(class_weight={0: 0.7902311382468382,
                                                           1: 0.7444535743631882,
                                                           2: 1.0909090909090908,
                                                           3: 1.2569367369589346,
                                                           4: 1.6743670301238218,
                                                           5: 0.9760827407886231,
                                                           6: 0.9458189790165988},
                                             random_state=42))],
                 n_jobs=-1)

* 모델 1, 모델 2-1, 모델 2-2를 다 만들었다.

* 이제 테스트 데이터를 가져와서 예측을 해볼까요?

<h1> 테스트 데이터를 이용하여 예측을 해봐요 </h1>

In [60]:
test_X = pd.read_csv('/content/drive/MyDrive/미니프로젝트5차part2_Kaggle/test.csv')

In [61]:
test_X.shape

(13234, 8)

In [62]:
test_X.columns

Index(['Unnamed: 0', 'timestamp', 'A_x', 'A_y', 'A_z', 'B_x', 'B_y', 'B_z'], dtype='object')

In [63]:
test_X['timestamp'] = pd.to_datetime(test_X['timestamp']) # 'timestamp' 변수의 데이터타입을 datetime으로 변경

In [64]:
# Timestamp 열의 각 구성 요소를 추출
test_X['Year'] = test_X['timestamp'].dt.year.astype(str)
test_X['Month'] = test_X['timestamp'].dt.month.astype(str).str.zfill(2)
test_X['Day'] = test_X['timestamp'].dt.day.astype(str).str.zfill(2)
test_X['Hour'] = test_X['timestamp'].dt.hour.astype(str).str.zfill(2)
test_X['Minute'] = test_X['timestamp'].dt.minute.astype(str).str.zfill(2)
test_X['Second'] = test_X['timestamp'].dt.second.astype(str).str.zfill(2)
test_X['MicroSecond'] = (test_X['timestamp'].dt.microsecond // 1000).astype(str).str.zfill(3)

# 각 구성 요소를 하나의 정수로 결합
test_X['Combined'] = (test_X['Year'] + test_X['Month'] + test_X['Day'] + test_X['Hour'] + test_X['Minute'] + test_X['Second'] + test_X['MicroSecond']).astype(int)

In [65]:
# 'Year', 'Month' ~ 'MicroSecond' 변수의 데이터타입을 int로 변환
test_X['Year'] = test_X['Year'].astype(int)
test_X['Month'] = test_X['Month'].astype(int)
test_X['Day'] = test_X['Day'].astype(int)
test_X['Hour'] = test_X['Hour'].astype(int)
test_X['Minute'] = test_X['Minute'].astype(int)
test_X['Second'] = test_X['Second'].astype(int)
test_X['MicroSecond'] = test_X['MicroSecond'].astype(int)

In [66]:
# test 데이터에 'Unamed : 0', 'timestamp' 변수를 삭제한다.
test_X.drop(['Unnamed: 0', 'timestamp'], axis=1, inplace=True)

* 앞서 표준화를 했었는데 이를 테스트 데이터에도 적용합시다.

In [67]:
columns = test_X.columns

In [68]:
test_X = pd.DataFrame(sds.transform(test_X), columns = columns) # 저는 2차원 넘파이 배열로 만들기 싫어서 데이터프레임으로 만들겠습니다.

In [69]:
test_X.head() # 표준화가 됐는지 확인한다.

,A_x,A_y,A_z,B_x,B_y,B_z,Year,Month,Day,Hour,Minute,Second,MicroSecond,Combined
0,-0.269818,-0.174250,0.383124,-0.247905,-0.194147,-0.559838,-0.676130,0.0,-0.676130,-0.681911,-1.573525,0.762410,0.724493,-0.676131
1,0.004351,0.702839,-1.358300,-0.614279,-0.121894,-0.060518,-0.676130,0.0,-0.676130,-0.681911,-0.135637,0.531308,0.170564,-0.676131
2,-0.744034,0.535107,1.245872,-0.100611,0.183875,-0.206172,-0.676130,0.0,-0.676130,-0.681911,-1.214053,-0.335325,1.416904,-0.676131
3,-0.067212,-0.177505,0.623843,-0.238546,0.381947,1.093860,1.479004,0.0,1.479004,0.958363,1.404959,0.877961,0.309046,1.479005
4,-0.335693,0.320199,-0.218778,0.697492,-0.756412,0.987876,1.479004,0.0,1.479004,-0.681911,1.456312,0.184655,-0.383365,1.479004


* 모델 1를 넣어봅시다.

In [70]:
model1_predict = list(model1.predict(test_X))

print('0으로 예측한 개수 : ', model1_predict.count(0))
print('1로 예측한 개수 : ', model1_predict.count(1))

0으로 예측한 개수 :  4904
1로 예측한 개수 :  8330


* model1_predict를 가지고 0이냐 1이냐에 따라서 모델 2-1, 모델 2-2를 통해 최종 예측을 시도한다.

In [71]:
# 모델 2-1를 통해 최종 예측 하는 함수
def static_predict(idx):
    result = model2_1.predict(test_X.iloc[idx].values.reshape(1, -1)) # Series를 2차원 넘파이 배열로 변환한다. 왜냐하면 머신러닝, 딥러닝 입력 형태가 데이터프레임이나 2차원 넘파이 배열이기 떄문
    return result[0]

In [72]:
# 모델 2-2를 통해 최종 예측 하는 함수
def dynamic_predict(idx):
    result = model2_2.predict(test_X.iloc[idx].values.reshape(1, -1)) # Series를 2차원 넘파이 배열로 변환한다. 왜냐하면 머신러닝, 딥러닝 입력 형태가 데이터프레임이나 2차원 넘파이 배열이기 떄문
    return result[0]

In [73]:
# 최종 예측된 것을 담는 리스트
model2_predict = []

* 모델 1 -> 모델 2-1
  *  최종 예측된 0, 1, 2, 3을 각각 6, 7, 8, 11으로 변환한다.

* 모델 1 -> 모델 2-2
  * 최종 예측된 0, 1, 2, 3, 4, 5, 6를 각각 1, 2, 3, 4, 5, 9, 10으로 변환한다.

In [74]:
# model1_predict 된 것을 하나씩 for문으로 돌린다.
for idx, m1p in enumerate(model1_predict):
    if m1p == 0:
        result = static_predict(idx)
        if result == 0:
            model2_predict.append(6)
        elif result == 1:
            model2_predict.append(7)
        elif result == 2:
            model2_predict.append(8)
        else:
            model2_predict.append(11)
    else:
        result = dynamic_predict(idx)
        if result == 0:
            model2_predict.append(1)
        elif result == 1:
            model2_predict.append(2)
        elif result == 2:
            model2_predict.append(3)
        elif result == 3:
            model2_predict.append(4)
        elif result == 4:
            model2_predict.append(5)
        elif result == 5:
            model2_predict.append(9)
        else:
            model2_predict.append(10)

<h1> Kaggle 제출 형태로 만든다 </h1>

In [75]:
submit = pd.DataFrame({
    'id' : list(range(0, 13234, 1)),
    'label' : model2_predict
})

In [76]:
submit['label'].value_counts()

2     1612
8     1539
1     1477
7     1406
10    1359
11    1269
9     1192
3     1028
4      961
5      701
6      690
Name: label, dtype: int64

In [77]:
# csv 파일로 저장
submit.to_csv('/content/drive/MyDrive/미니프로젝트5차part2_Kaggle/predict.csv', index=False)

* 영렬님 최종 Kaggle 제출 파일을 가져온다. 정확도 0.99****을 찍으셨기 떄문에 정답이라 봐도 무방할 것이다.

In [78]:
youngYul_submit = pd.read_csv('/content/drive/MyDrive/미니프로젝트5차part2_Kaggle/영렬님 help/prediction_results (16)(영렬님 최종 예측 label 빈도수).csv')

In [79]:
youngYul_submit['label'].value_counts()

2     1617
8     1539
1     1474
7     1406
10    1355
11    1265
9     1192
3     1044
4      946
6      703
5      693
Name: label, dtype: int64